In [177]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import Normalizer

In [178]:
train_set = pd.read_csv("q4-train-dataset.csv")
test_set = pd.read_csv("q4-test-dataset.csv")
train_set = train_set.values
test_set = test_set.values
trainLabel = train_set[:,29:30]
trainFeatures = train_set[:, :-1]
testLabel = test_set[:,29:30]
testFeatures = test_set[:,:-1]


In [182]:
def LogStocReg(train_set, iterCount, featSize, learnRate):
    weights = np.random.normal(0, 0.01, featSize)
    #logistic_weights_list = []
    #for learnRate in learnRates:
    trainLabel = train_set[:,29:30]
    trainFeatures = train_set[:, :-1]
    trainFeatures = trainFeatures - trainFeatures.mean(axis=0)
    trainFeatures = trainFeatures / np.abs(trainFeatures).max(axis=0)
    for i in range(iterCount):
        w0 = 0
        for j in range(800):
            ss = 1/(1+np.exp(-np.dot(trainFeatures[j], weights)))
            w0 += np.dot(trainFeatures[j],(trainLabel[j][0] - ss))
            if(j != 0 and j%1 == 0):
                weights += learnRate * w0
                #print("learn rate: " +str(learnRate)+" w0: "+ str(w0) )
                w0 = 0
            elif(j == 799):
                weights += learnRate * w0
                w0 = 0
        #logistic_weights_list.append(weights)
    #return logistic_weights_list
    return weights

def LogFullReg(train_set, iterCount, featSize, learnRate):
    weights = np.random.normal(0, 0.01, featSize)
    #logistic_weights_list = []
    #for learnRate in learnRates:
    trainLabel = train_set[:,29:30]
    trainFeatures = train_set[:, :-1]
    trainFeatures = trainFeatures - trainFeatures.mean(axis=0)
    trainFeatures = trainFeatures / np.abs(trainFeatures).max(axis=0)
    for i in range(iterCount):
        w0 = 0
        for j in range(800):
            ss = 1/(1+np.exp(-np.dot(trainFeatures[j], weights)))
            w0 += np.dot(trainFeatures[j],(trainLabel[j][0] - ss))
            if(j != 0 and j%14000 == 0):
                weights += learnRate * w0
                #print("learn rate: " +str(learnRate)+" w0: "+ str(w0) )
                w0 = 0
            elif(j == 799):
                weights += learnRate * w0
                w0 = 0
        #logistic_weights_list.append(weights)
    #return logistic_weights_list
    return weights

def LogRegMinBatch(train_set, iterCount, featSize, learnRate):
    weights = np.random.normal(0, 0.01, featSize)
    trainLabel = train_set[:,29:30]
    trainFeatures = train_set[:, :-1]
    trainFeatures = trainFeatures - trainFeatures.mean(axis=0)
    trainFeatures = trainFeatures / np.abs(trainFeatures).max(axis=0)
    for i in range(iterCount):
        w0 = 0
        for j in range(800):
            w0 += np.dot(trainFeatures[j],(trainLabel[j][0] - 1/(1+np.exp(-np.dot(trainFeatures[j], weights)))))
            if(j != 0 and j%32 == 0):
                weights += learnRate * w0
                w0 = 0
            elif(j == 799):
                weights += learnRate * w0
                w0 = 0
    return weights
    

def findResult(weights, test_feature, test_label):
    #result = []
    #for weight in weights:
    prediction = np.dot(test_feature, weights)
    prediction[prediction >= 0] = 1
    prediction[prediction < 0] = 0
    num = 0
    TruePositive = 0
    TrueNegative = 0
    FalsePositive = 0
    FalseNegative = 0
    print(len(prediction))
    for i in range(len(prediction)):
        if(prediction[i] == test_label[i]):
            if(prediction[i]==0):
               TrueNegative += 1 
            elif(prediction[i]==1):
                TruePositive += 1
            num += 1
        else:
            if(prediction[i]==0):
               FalseNegative += 1 
            elif(prediction[i]==1):
                FalsePositive += 1
        
    print("True Positive: "+str(TruePositive))
    print("True Negative: "+str(TrueNegative))
    print("False Positive: "+str(FalsePositive))
    print("False Positive: "+str(FalseNegative))
    showValues(TruePositive, TrueNegative, FalsePositive, FalseNegative)
    #result.append(num / len(test_label) * 100)  
    return num / len(test_label) * 100

def showConfMatrix(truePos, trueNeg, falsePos, falseNeg):
    
    print("REAL->     Pos.        Neg.   ")
    print("     |--------------------------|")
    print("Pos. |    "+str(truePos)+"        "+ str(falsePos)+"           |")
    print("Neg. |    "+str(falseNeg)+"        "+ str(trueNeg)+"           |")
    print("     |--------------------------|")
    
def showValues(truePos, trueNeg, falsePos, falseNeg):
    recall = truePos/(truePos+falseNeg)
    npv = trueNeg/(trueNeg+falseNeg)
    fpr = falsePos/(falsePos+trueNeg)
    fdr = falsePos/(truePos+falsePos)
    precision = truePos/(truePos+falsePos)
    try:
        f1 = (2*(truePos/(truePos+falsePos)))/((truePos/(truePos+falsePos))+(truePos/(truePos+falseNeg)))
    except (ZeroDivisionError):
        f1 = 0
        
    try:
        f2 = (5*(truePos/(truePos+falsePos)))/ (4*(truePos/(truePos+falsePos))+(truePos/(truePos+falseNeg)))
    except (ZeroDivisionError):
        f2 = 0
        
    
    print("Precision: " + str(precision))  
    print("Recall: " + str(recall))
    print("FDR: " + str(fdr))
    print("NPV: " + str(npv))
    print("FPR: " + str(fpr))
    print("f1: " + str(f1))
    print("f2: " + str(f2))
    showConfMatrix(truePos, trueNeg, falsePos, falseNeg)
    
def findBestStep(fin1, fin2, fin3, fin4, fin5, rates):
    if(fin3 == max(fin1, fin2, fin3, fin4, fin5)):
        return rates[2]
    elif(fin4 == max(fin1, fin2, fin3, fin4, fin5)):
        return rates[3]
    elif(fin5 == max(fin1, fin2, fin3, fin4, fin5)):
        return rates[4]
    elif(fin1 == max(fin1, fin2, fin3, fin4, fin5)):
        return rates[0]
    elif(fin2 == max(fin1, fin2, fin3, fin4, fin5)):
        return rates[1]

In [180]:
#label trial for full batch

test_set_label = test_set[:,29:30]
test_set_feature = test_set[:, :-1]
test_set_feature = test_set_feature - test_set_feature.mean(axis=0)
test_set_feature = test_set_feature / np.abs(test_set_feature).max(axis=0)
#for i in range(10 ,100, 10):
learn_rates = [0.00001, 0.0001, 0.001, 0.01, 0.1]
weights = LogFullReg(train_set, 1000, 29, 0.00001)
acc = findResult(weights, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.00001 in 1000 iterations: ")
print(str(acc))
weights1 = LogFullReg(train_set, 1000, 29, 0.0001)
acc1 = findResult(weights1, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.0001 in 1000 iterations: ")
print(str(acc1))
weights2 = LogFullReg(train_set, 1000, 29, 0.001)
acc2 = findResult(weights2, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.001 in 1000 iterations: ")
print(str(acc2))
weights3 = LogFullReg(train_set, 1000, 29, 0.01)
acc3 = findResult(weights3, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.01 in 1000 iterations: ")
print(str(acc3))
weights4 = LogFullReg(train_set, 1000, 29, 0.1)
acc4 = findResult(weights4, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.1 in 1000 iterations: ")
print(str(acc4))

best_rate =  findBestStep(acc, acc1, acc2, acc3, acc4, learn_rates)
print("Best learn rate is: " + str(best_rate))

126
True Positive: 47
True Negative: 63
False Positive: 0
False Positive: 16
Precision: 1.0
Recall: 0.746031746031746
FDR: 0.0
NPV: 0.7974683544303798
FPR: 0.0
f1: 1.1454545454545455
f2: 1.0535117056856187
REAL->     Pos.        Neg.   
     |--------------------------|
Pos. |    47        0           |
Neg. |    16        63           |
     |--------------------------|
Test Accuracy for learn rates of 0.00001 in 1000 iterations: 
87.3015873015873
126
True Positive: 48
True Negative: 63
False Positive: 0
False Positive: 15
Precision: 1.0
Recall: 0.7619047619047619
FDR: 0.0
NPV: 0.8076923076923077
FPR: 0.0
f1: 1.135135135135135
f2: 1.05
REAL->     Pos.        Neg.   
     |--------------------------|
Pos. |    48        0           |
Neg. |    15        63           |
     |--------------------------|
Test Accuracy for learn rates of 0.0001 in 1000 iterations: 
88.09523809523809
126
True Positive: 49
True Negative: 62
False Positive: 1
False Positive: 14
Precision: 0.98
Recall: 0.77777

In [181]:
#iteration number test for full batch
for i in range(10,110,10):
    test_set_label = test_set[:,29:30]
    test_set_feature = test_set[:, :-1]
    test_set_feature = test_set_feature - test_set_feature.mean(axis=0)
    test_set_feature = test_set_feature / np.abs(test_set_feature).max(axis=0)
    weights4 = LogFullReg(train_set, i*10, 29, 0.01)
    acc5 = findResult(weights4, test_set_feature, test_set_label)
    print("Test Accuracy for learn rates of 0.1 in "+ str(i*10) +" iterations: ")
    print(str(acc5))

126
True Positive: 49
True Negative: 62
False Positive: 1
False Positive: 14
Precision: 0.98
Recall: 0.7777777777777778
FDR: 0.02
NPV: 0.8157894736842105
FPR: 0.015873015873015872
f1: 1.1150442477876106
f2: 1.0430463576158941
REAL->     Pos.        Neg.   
     |--------------------------|
Pos. |    49        1           |
Neg. |    14        62           |
     |--------------------------|
Test Accuracy for learn rates of 0.1 in 100 iterations: 
88.09523809523809
126
True Positive: 49
True Negative: 62
False Positive: 1
False Positive: 14
Precision: 0.98
Recall: 0.7777777777777778
FDR: 0.02
NPV: 0.8157894736842105
FPR: 0.015873015873015872
f1: 1.1150442477876106
f2: 1.0430463576158941
REAL->     Pos.        Neg.   
     |--------------------------|
Pos. |    49        1           |
Neg. |    14        62           |
     |--------------------------|
Test Accuracy for learn rates of 0.1 in 200 iterations: 
88.09523809523809
126
True Positive: 50
True Negative: 62
False Positive: 1
Fals

In [176]:
#mini batch
test_set_label = test_set[:,29:30]
test_set_feature = test_set[:, :-1]
test_set_feature = test_set_feature - test_set_feature.mean(axis=0)
test_set_feature = test_set_feature / np.abs(test_set_feature).max(axis=0)
weights4 = LogRegMinBatch(train_set, 1000, 29, best_rate)
acc5 = findResult(weights4, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.1 in "+ str(1000) +" iterations: ")
print(str(acc5))

126
True Positive: 49
True Negative: 62
False Positive: 1
False Positive: 14
Recall: 0.7777777777777778
FDR: 0.02
NPV: 0.8157894736842105
FPR: 0.015873015873015872
f1: 0
f2: 0
REAL->     Pos.        Neg.   
     |--------------------------|
Pos. |    49        1           |
Neg. |    14        62           |
     |--------------------------|
Test Accuracy for learn rates of 0.1 in 1000 iterations: 
88.09523809523809


In [183]:
#stochastic batch
test_set_label = test_set[:,29:30]
test_set_feature = test_set[:, :-1]
test_set_feature = test_set_feature - test_set_feature.mean(axis=0)
test_set_feature = test_set_feature / np.abs(test_set_feature).max(axis=0)
weights5 = LogStocReg(train_set, 1000, 29, best_rate)
acc6 = findResult(weights5, test_set_feature, test_set_label)
print("Test Accuracy for learn rates of 0.1 in "+ str(1000) +" iterations: ")
print(str(acc6))

126
True Positive: 49
True Negative: 62
False Positive: 1
False Positive: 14
Precision: 0.98
Recall: 0.7777777777777778
FDR: 0.02
NPV: 0.8157894736842105
FPR: 0.015873015873015872
f1: 1.1150442477876106
f2: 1.0430463576158941
REAL->     Pos.        Neg.   
     |--------------------------|
Pos. |    49        1           |
Neg. |    14        62           |
     |--------------------------|
Test Accuracy for learn rates of 0.1 in 1000 iterations: 
88.09523809523809
